# **Project Goals**

The goal of the project would be as follows:

1: Write your own code (not an existing repo on the internet) that performs single shot ptychography using structured illumination as demonstrated in the papers by Levitan et al.

2: answer the following question:

a. What happens if the illumination (=probe) field has zero crossings? Is this a problem for the method? Do zero crossing result in non-measured points?

b. suppose the method is simulated using 2x2 upsampling (that is, 2x2 probe pixels determine the average phase in a 1x1 super-pixel inside the object). Is it required that there is a relative phase shift between the 2x2 pixels inside the probe? Or can the modulation be amplitude-only (that is no phase shift, but an intensity variation)

c. What is the minimum amount of upsampling needed?

d. Is there a class of illumination functions for which the method fails?

e. Try implementing regularization to the method. Please include L1, L2, and Total variation regularization.

# **Project Ptychography**